In [1]:
import sys
#print(sys.path)
sys.path.append("/Users/raroito/PycharmProjects/facial_expression_recognition/src/")

from utils import ModelImporter, Preprocessing
import torch
import pandas as pd
import numpy as np

# Analize Test Data Results

In [2]:
n_classes = 7
n_epochs = 100
learning_rate = 0.00001
batch_size = 32

model_name = f'cnn_simple_reduced_{learning_rate}_{batch_size}_{n_epochs}_{n_classes}'

In [3]:
pre = Preprocessing('fer2013')
pre.load_data(filename='test_public_norm.csv', name='test')

X_df = pre.get(name='test').drop(columns=['emotion'])
y_df = pre.get(name='test')['emotion']

In [4]:
dtype = torch.float
device = torch.device("cpu")

m_importer = ModelImporter('fer2013_reduced')
model = m_importer.load_nn_model(model_name)

['cnn_simple_reduced_1e-05_32_100_7', 1, 7, [48, 48], 15, 2]
load model CnnSimple(
  (drop_hidden): Dropout(p=0.5)
  (drop_visible): Dropout(p=0.2)
  (conv1): Conv2d(1, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchnorm1): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8640, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=7, bias=True)
)


In [5]:
X_test = model.reshape_data(torch.tensor(X_df.values, device=device, dtype=dtype))
y_test = torch.tensor(y_df.values, device=device, dtype=torch.long)

In [6]:
y_pred_prob = model(X_test)

In [7]:
y_pred = y_pred_prob.argmax(1)

In [8]:
y_pred_prob

tensor([[ 2.1990, -4.7324,  1.4614,  ...,  0.7299,  0.9090, -0.3925],
        [ 0.7832, -2.4167,  0.1636,  ...,  1.6757, -0.5131, -0.2876],
        [ 0.2617, -1.6025,  0.1360,  ...,  0.7132, -0.2929,  0.5698],
        ...,
        [ 0.6945, -2.7629,  0.4775,  ...,  1.3964, -0.0211,  1.1371],
        [ 0.7520, -2.7923, -0.0275,  ...,  0.7201, -1.2487,  0.9701],
        [ 0.8740, -2.4512, -0.3211,  ...,  1.2325, -0.1721,  0.5034]],
       grad_fn=<AddmmBackward>)

In [9]:
y_pred

tensor([0, 4, 4,  ..., 4, 6, 4])

In [10]:
accuracy_soft = (y_pred == y_test).float().mean()
accuracy_soft

tensor(0.4224)

In [12]:
#try somee convertions for the f1 score
from sklearn import metrics
metrics.f1_score(y_test.numpy(), y_pred.numpy())

ValueError: Target is multiclass but average='binary'. Please choose another average setting.

In [11]:
results = pd.DataFrame()

In [12]:
results['y_test'] = y_test
results['y_pred'] = y_pred
results['right'] = (y_pred == y_test).float()
results['ones'] = 1

In [13]:
results.tail()

,y_test,y_pred,right,ones
3584,4,0,0.0,1
3585,3,3,1.0,1
3586,4,4,1.0,1
3587,4,6,0.0,1
3588,4,4,1.0,1


In [14]:
report = results.groupby('y_test').sum()
report

,y_pred,right,ones
y_test,,,
0,1502,77.0,467
1,145,0.0,56
2,1742,83.0,496
3,2871,674.0,895
4,2336,226.0,653
5,1725,226.0,415
6,2571,230.0,607


In [15]:
report['%'] = report['right']/report['ones']
report

,y_pred,right,ones,%
y_test,,,,
0,1502,77.0,467,0.164882
1,145,0.0,56,0.000000
2,1742,83.0,496,0.167339
3,2871,674.0,895,0.753073
4,2336,226.0,653,0.346095
5,1725,226.0,415,0.544578
6,2571,230.0,607,0.378913


In [ ]:
#class 3 is somhow well recognized (also is the class with more samples) 
#whereas not a single sample of class 2 was properly detected

# Analize the Training Dataset

In [16]:
pre = Preprocessing('fer2013')
pre.load_data(filename='train_norm.csv', name='train')

X_df = pre.get(name='train').drop(columns=['emotion'])
y_df = pre.get(name='train')['emotion']

In [17]:
X_train= model.reshape_data(torch.tensor(X_df.values, device=device, dtype=dtype))
y_train = torch.tensor(y_df.values, device=device, dtype=torch.long)

In [18]:
y_pred_prob = model(X_train)

In [19]:
y_pred = y_pred_prob.argmax(1)

In [20]:
y_pred_prob

tensor([[ 0.0899, -2.2774,  0.1053,  ...,  1.3442, -0.4815,  0.3664],
        [ 1.3614, -2.6592,  0.1181,  ...,  0.4263, -1.0753,  0.3331],
        [-0.1083, -2.1375, -0.0544,  ...,  0.1629,  0.4555,  0.0800],
        ...,
        [ 0.1332, -1.0554,  0.7759,  ...,  0.5254, -0.0467,  0.0384],
        [-0.7174, -1.8970, -0.0522,  ..., -0.7629,  0.7101,  1.0902],
        [-0.0928, -1.3106,  0.7878,  ...,  0.1169,  0.9926, -0.1449]],
       grad_fn=<AddmmBackward>)

In [21]:
y_pred

tensor([4, 0, 5,  ..., 2, 6, 5])

In [22]:
y_train

tensor([0, 0, 2,  ..., 4, 0, 4])

In [23]:
results_train['y_train'] = y_train
results_train['y_pred'] = y_pred
results_train['right'] = (y_pred == y_train).float()
results_train['ones'] = 1

NameError: name 'results_train' is not defined

In [ ]:
report_train = results_train.groupby('y_train').sum()
report_train

In [ ]:
report_train['%'] = report_train['right']/report_train['ones']
report_train

In [ ]:
#similar results with the train data as with the 